In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from inxss import SirenNet

from sklearn.model_selection import train_test_split

In [2]:
from inxss import SpectrumDataset, SpecNeuralRepr
from torch.utils.data import DataLoader

In [3]:
spec_dataset = SpectrumDataset(
        '/pscratch/sd/z/zhantao/inxs_steering/SpinW_data/summarized_AFM_data_2023Aug01.pt',
        num_wq=10000
    )

In [4]:
train_idx, val_test_idx = train_test_split(np.arange(len(spec_dataset)), test_size=0.2, random_state=42)
val_idx, test_idx = train_test_split(val_test_idx, test_size=0.5, random_state=42)

train_loader = DataLoader([spec_dataset[i] for i in train_idx], batch_size=10, shuffle=True)
val_loader = DataLoader([spec_dataset[i] for i in val_idx], batch_size=10, shuffle=False)
test_loader = DataLoader([spec_dataset[i] for i in test_idx], batch_size=10, shuffle=False)

In [5]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar

model = SpecNeuralRepr()

In [6]:
checkpoint_callback = ModelCheckpoint(
    save_on_train_epoch_end=False, save_last=True, save_top_k=1, monitor="val_loss"
)

trainer = pl.Trainer(
    max_epochs=10000, accelerator="gpu",
    callbacks=[TQDMProgressBar(refresh_rate=10), checkpoint_callback],
    log_every_n_steps=1, devices=1,
    enable_checkpointing=True,
    default_root_dir='/pscratch/sd/z/zhantao/inxs_steering'
)

trainer.fit(model, train_loader, val_loader)

/global/homes/z/zhantao/.conda/envs/inxs_steering/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /global/homes/z/zhantao/.conda/envs/inxs_steering/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/global/homes/z/zhantao/.conda/envs/inxs_steering/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /global/homes/z/zhantao/.conda/envs/inxs_steering/li ...
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/z/zhantao/.conda/envs/inxs_steering/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/global/homes/z/zhantao/.conda/envs/inxs_steering/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 5122:  62%|██████▎   | 10/16 [00:00<00:00, 31.55it/s, v_num=1.35e+7]

/global/homes/z/zhantao/.conda/envs/inxs_steering/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
